##### Example 1

In [4]:
import gymnasium as gym

In [5]:
env = gym.make('CartPole-v1')

In [20]:
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>

In [21]:
import torch.distributed.rpc as rpc

Create an Observer class in which the method run_episode runs an episode with a maximum of 69 steps using `agent.select_action(observer_id, state)` via its RRef

**Hint**: Taking an action returns: `state, reward, done, _`

In [22]:
class Observer:
    def __init__(self, env):
        self.id = rpc.get_worker_info().id
        self.env = env
    
    def run_episode(self, agent_rref):
        state, _ = self.env.reset()
        
        for _ in range(69):
            # send the state to the agent to get an action
            action = agent_rref.rpc_sync().select_action(self.id, state)
            
            # apply the action to the environment, and get the reward
            state, reward, done, _ = self.env.step(action)
            
            if done: break # could be ignore

In [ ]:
observer = Observer(env)

In [ ]:
observer.run_episode(agent_rref)

##### Example 2

In [29]:
class Agent:
    pass

Add `Agent`'s rref in the `Agent.id` attribute during initialization

In [30]:
from torch.distributed.rpc import RRef

In [31]:
class Agent:
    def __init__(self):
        self.id = RRef(self)

In [32]:
Agent

__main__.Agent

##### Example 2

In [3]:
world_size = 4

In [4]:
world_size

4

Launch four new processes and initialize RPC. Each process should have the name agent_rank.

Hint: Set `MASTER_ADDR` to `localhost` and `MASTER_PORT` to `29500`.

In [5]:
import os
import torch.multiprocessing as mp

In [6]:
AGENT_NAME = "agent_{}"

In [7]:
AGENT_NAME.format(1)

'agent_1'

In [8]:
def run_worker(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'
    
    rpc.init_rpc(AGENT_NAME.format(rank), rank=rank, world_size=world_size)


In [ ]:
for rank in range(world_size):
    process = mp.Process(target=run_worker, args=(rank, world_size))
    process.start()

##### Example 3

In [28]:
ob_rrefs = []

In [32]:
type(ob_rrefs)

list

Invoke the `run` function asynchronously on all objects in `ob_rrefs`. Then, wait for all of them to complete.     

In [33]:
import torch.distributed.rpc as rpc

In [34]:
futs = []

In [ ]:
for ob_rref in ob_rrefs:
    futs.append(rpc.rpc_async(
        to=ob_rref.owner(),
        func=ob_rref.rpc_sync().run,
    ))

In [ ]:
for fut in futs:
    fut.wait()

##### Example 4